# Data Mining - A2Z Insurance

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
import matplotlib.cm
import scipy
from scipy import stats as st
import itertools
from math import ceil, floor
from random import sample
from numpy.random import uniform

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set_theme()

# Preprocessing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import squareform, pdist
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.neighbors import KDTree
from collections import Counter

# SOM
import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView

# Sklearn clustering algorithms
from sklearn.cluster import AgglomerativeClustering, KMeans, MeanShift, DBSCAN
from scipy.cluster.hierarchy import dendrogram

# Other clustering algorithms
from kmodes.kprototypes import KPrototypes
import kmedoids

# Clusters evaluation
from sklearn.metrics import silhouette_score, silhouette_samples

# Gower distance
import gower

# Multidimensional visualization
from sklearn.manifold import TSNE

# Feature importance
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_sas('a2z_insurance.sas7bdat')

## Data exploration <a class="anchor" id="exploration"></a>

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

We can see some missing values as well as strange values in some columns such as *FirstPolYear* and *BirthYear*. It makes more sense to define *CustID* as the index of the dataframe. Also, *EducDeg* is defined as a byte variable and should be converted to integer

In [ ]:
df["CustID"] = df["CustID"].astype(int)

df.set_index("CustID",inplace=True) # Turn CustID into index

Let's check the distributions of the categorical features

In [ ]:
df['EducDeg'].value_counts()

In [ ]:
df['GeoLivArea'].value_counts()

In [ ]:
df['Children'].value_counts()

### Types conversion <a class="anchor" id="types"></a>

In [ ]:
df["EducDeg"].replace({b'1 - Basic':1, b'2 - High School':2, b'3 - BSc/MSc':3, b'4 - PhD':4}, inplace = True)

### Duplicates <a class="anchor" id="duplicates"></a>

In [ ]:
df_discarded = df[df.duplicated()].copy()

In [ ]:
df_discarded

In [ ]:
df.duplicated().sum()

In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.duplicated().sum()

### Coherence check <a class="anchor" id="coherence"></a>

We found some strange values:
- FirstPolYear: Max > 2016
- FirtPolYear < BirthYear



In [ ]:
# FirstPolYear > 2016
df[df['FirstPolYear'] > 2016]

In [ ]:
df_discarded = pd.concat([df_discarded, df[df['FirstPolYear'] > 2016]])

In [ ]:
# Only one observation, let's drop it
df.drop(9295, axis = 0, inplace = True)

In [ ]:
# FirstPolYear < BirthYear
df[df["FirstPolYear"] < df["BirthYear"]]

There are a lot of observations in this situation. Given we do not want to drop them as it would result in a great loss of data, we found an explanation:
If FirstPolYear < BirthYear then FirstPolYear is set to BirthYear, meaning that newborns/children were insured by parents and then started paying their own insurance and by the time the insurance was contracted by the parent, the FirstPolYear registered was the parent's.


In [ ]:
df.loc[df["FirstPolYear"]<df["BirthYear"],"FirstPolYear"] = df.loc[df["FirstPolYear"]<df["BirthYear"],"BirthYear"]

df[df["FirstPolYear"]<df["BirthYear"]]

### Data visualization <a class="anchor" id="visualization"></a>

In [ ]:
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal', 'CustMonVal', 'ClaimsRate', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']

non_metric_features = ['EducDeg', 'GeoLivArea', 'Children']

__1.__ **Distribution of metric features**

In [ ]:
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(25, 11))

for ax, feat in zip(axes.flatten(), metric_features):
    sns.histplot(df[feat], bins=10, kde=True , legend=False, ax = ax)

plt.suptitle('Distribution of metric features', fontsize = 20)

# We can see a lot of outliers

__2.__ **Spearman Correlation matrix between numerical + ordinal features**

In [ ]:
fig = plt.figure(figsize=(15, 15))

# Given we are using Spearman Correlation, it also makes sense to include the ordinal variable
feats = metric_features + ['EducDeg']

corr = np.round(df[feats].corr(method="spearman"), decimals=2)

# Only plot the correlation values if the value surpasses a 0.5 threshold
annotations = np.where(np.absolute(corr.values) >= 0.5, corr.values, np.full(corr.shape,""))
sns.heatmap(data=corr, annot=annotations, cmap='BrBG', fmt='s', center=0, square=True, linewidths=.5)

plt.suptitle("Numerical + Ordinal features Spearman Correlation Matrix", fontsize=20, y=0.9)
plt.show()

*ClaimsRate* is highly correlated with *CustMonVal* as well as *BirthYear* with *MonthSal*. From now on, we will only be using one or the other for both cases.

__3.__ **Customers with negative CustMonVal spend more money in which premiums?**

In [ ]:
# Negative CMV means that the anual profit from the customer is so low that does not compensate the aquisition cost
df_cmv = df.copy()

df_cmv['CMV_pos'] = df_cmv['CustMonVal'].apply(lambda x: -1 if x < 0 else 1)

df_cmv['CMV_pos'].value_counts()

In [ ]:
df_cmv.groupby('CMV_pos')[['PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']].mean().T.plot.bar(figsize = (10, 7), edgecolor = 'white', color = {1: '#224758', -1 : '#bf441f'})
plt.legend(title = 'CMV')
plt.ylabel('Mean spent')
plt.title('Money spent in each premium VS profitable/non-profitable client', fontsize = 20)

People with negative CMV spend a lot more money in Motor and less in the rest of the insurances than people with positive CMV. This probably means that motor is a riskier type of insurance, which can lead to more economic losses for the company

__4.__ **Money spent in premiums VS Number of years as a customer**

In [ ]:
plt.figure(figsize = (20,5))

premiums = ['PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']

total_premiums = np.sum(df[premiums], axis = 1)

sns.barplot(x = df['FirstPolYear'], y = total_premiums)

plt.suptitle("Total of premiums across customers' first policy year", fontsize = 20)

# Similar total premiums per nr of years as customer. Looks like most recent clients are spending a little more.

__5.__ **Amount spent in each premium VS Age**

In [ ]:
df_discretized = df.copy()

df_discretized['birthyear_binned']=pd.cut(x=df['BirthYear'], bins=[1920,1940,1960,1980, 2001])

df_discretized.groupby('birthyear_binned')[premiums].mean().plot.bar(stacked = False, cmap = cm.Set3, figsize = (12, 7))

plt.xlabel("Birth_Year")
plt.ylabel('Mean Premiums')
plt.title("Age Vs Type of premiums")

plt.xticks(rotation = 0)

# People with different ages spend money in different insurances

__6.__ **Amount spent in each premium VS GeoLivArea**

In [ ]:
df.groupby('GeoLivArea')[premiums].mean().plot.bar(stacked = False, cmap = cm.Set1, figsize = (12, 7))

plt.ylabel('Mean Premiums')
plt.title("GeolIvArea Vs Premiums spent on each insurance")

# This variable does not seem to have a good discriminant power when combined with the insurances

__7.__ **Distribution of number of insurances**

In [ ]:
df_nr = df.copy()

zeros = (df.loc[:, ['PremMotor', 'PremHousehold', 'PremLife', 'PremHealth', 'PremWork']] == 0).sum(1)
nans = (df.loc[:, ['PremMotor', 'PremHousehold', 'PremLife', 'PremHealth', 'PremWork']].isna().sum(1))
df_nr['nr_insurances'] = 5 - (zeros + nans)

plt.figure(figsize = (10, 7))
plt.yscale('log') # log scale because there is a great discrepancy on the values
sns.countplot(data = df_nr, x ='nr_insurances', order = df_nr['nr_insurances'].value_counts().index.values[::-1])
plt.ylabel('log(number of customers)')

# A lot of people have paid for the 5 available insurances

## Data preprocessing <a class="anchor" id="preprocessing"></a>

### Outliers using IQR <a class="anchor" id="outliersIQR"></a>


In [ ]:
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))

for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(x=df[feat], ax=ax)
    
plt.suptitle("Metric features distributions before outliers removal", fontsize = 20)

In [ ]:
df.shape[0]

In [ ]:
df_outliers = df.copy()

df_no_outliers = df.copy()

def remove_outliers(condition, col):
  global df_no_outliers
  df_clean = df_no_outliers[condition]
  df_na = df_no_outliers[df_no_outliers[col].isna()]
  df_no_outliers = pd.concat([df_clean, df_na])

# Discard these values because they are not representative of the population
remove_outliers(df_no_outliers['BirthYear'] > 1900, 'BirthYear')
remove_outliers(df_no_outliers['MonthSal'] < 20000, 'MonthSal')
remove_outliers(df_no_outliers['CustMonVal'] > -2000, 'CustMonVal')
remove_outliers(df_no_outliers['ClaimsRate'] < 4, 'ClaimsRate')
remove_outliers(df_no_outliers['PremMotor'] < 2000, 'PremMotor')
remove_outliers(df_no_outliers['PremHousehold'] < 1500, 'PremHousehold')
remove_outliers(df_no_outliers['PremHealth'] < 400, 'PremHealth')
remove_outliers(df_no_outliers['PremLife'] < 300 , 'PremLife')
remove_outliers(df_no_outliers['PremWork'] < 300 , 'PremWork')

df_outliers.drop(df_no_outliers.index, inplace = True)

In [ ]:
print(f'Percentage of removed data: {np.round(((10296 - df_no_outliers.shape[0]) / 10296) * 100, 4)} %')

In [ ]:
df_discarded = pd.concat([df_discarded, df_outliers])
df_discarded.shape

In [ ]:
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))

for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(x=df_no_outliers[feat], ax=ax)

plt.suptitle("Metric features distributions after outliers removal", fontsize = 20)

### Missing values - Premiums <a class="anchor" id="missingPremiums"></a>

We have to fill these missing values at this point to do feature engineering before scaling. Later we will use KNN imputation to fill some of the other variables missing values, but before that we will need to scale the data.

We assumed that all the premiums that were not registered have value 0


In [ ]:
df_no_outliers[premiums] = df_no_outliers[premiums].fillna(0)

# for final reclassification of outliers
df_discarded[premiums] = df_discarded[premiums].fillna(0)

In [ ]:
df_no_outliers.isna().sum()

In [ ]:
df_discarded.isna().sum()

### Feature engineering <a class="anchor" id="featureEngineering"></a>

#### Total reversals <a class="anchor" id="reversals"></a>

Absolute value of the total amount of negative premiums.
Represents the value that the company owes the customer.

In [ ]:
df_no_outliers['total_reversals'] = np.abs(df_no_outliers[premiums][df_no_outliers[premiums] < 0].sum(1))

# for final reclassification of outliers
df_discarded['total_reversals'] = np.abs(df_discarded[premiums][df_discarded[premiums] < 0].sum(1))

Horizontal translation of the features to make them all positive

We are doing this so that we can apply transformations to the data later on to reduce skewness. Depite performing this translation, we ensure that we do not lose any information about the comparison of features among clients seeing that the relationships of "higher" and "lower" remain unchanged, only the values are altered. Additionally, the values will later be transformed into scaled values that do not have meaning on their own and they only have significance when compared to each other in the final step of our cluster analysis.

In [ ]:
df_no_outliers['PremMotor'] += abs(df_no_outliers['PremMotor'].min())
df_no_outliers['PremHousehold'] += abs(df_no_outliers['PremHousehold'].min())
df_no_outliers['PremHealth'] += abs(df_no_outliers['PremHealth'].min())
df_no_outliers['PremLife'] += abs(df_no_outliers['PremLife'].min())
df_no_outliers['PremWork'] += abs(df_no_outliers['PremWork'].min())

In [ ]:
df_no_outliers.describe()

#### Total premiums <a class="anchor" id="totalPremiums"></a>

In [ ]:
df_no_outliers['total_premiums'] = df_no_outliers[['PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']].sum(axis=1)

# for final reclassification of outliers
df_discarded['total_premiums'] = df_discarded[['PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']].sum(axis=1)

#### Premiums percentages <a class="anchor" id="premiumsPercentage"></a>
How much of the total spent does the client spent in each type of insurance

In [ ]:
for col in premiums:
  df_no_outliers[col[4:]+'Percent'] = df_no_outliers[col] / df_no_outliers['total_premiums']

  # for final reclassification of outliers
  df_discarded[col[4:]+'Percent'] = df_discarded[col] / df_discarded['total_premiums']

In [ ]:
df_no_outliers.head()

### Transforming skewed data <a class="anchor" id="skewed"></a>

We need to transform skewed data because there are some algorithms that assume a symetric and spherical distribution of variables

In [ ]:
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal',
       'CustMonVal', 'ClaimsRate', 'PremMotor', 'PremHousehold',
       'PremHealth', 'PremLife', 'PremWork', 'total_premiums', 'total_reversals',
        'MotorPercent', 'HouseholdPercent',
       'HealthPercent', 'LifePercent', 'WorkPercent']

fig, axes = plt.subplots(3, ceil(len(metric_features) / 3), figsize=(30, 30))

for ax, feat in zip(axes.flatten(), metric_features): # Notice the zip() function and flatten() method
    sns.histplot(df_no_outliers[feat],bins=10, kde=True , legend=False, ax = ax)

In [ ]:
df_no_outliers[metric_features].skew(axis = 0)

In [ ]:
sns.histplot(df_no_outliers['PremHousehold'] ** (1/3), bins = 10, kde=True , legend=False)

In [ ]:
sns.histplot(df_no_outliers['PremLife'] ** (1/3), bins = 10, kde=True , legend=False)

In [ ]:
sns.histplot(df_no_outliers['PremWork'] ** (1/3), bins = 10, kde=True , legend=False)

In [ ]:
sns.histplot(df_no_outliers['total_premiums'] ** (1/1.2), bins = 10, kde=True , legend=False)

In [ ]:
df_no_outliers['total_premiums'] = df_no_outliers['total_premiums'] ** (1/1.2)

df_no_outliers['PremHousehold'] = df_no_outliers['PremHousehold'] ** (1/3)
df_no_outliers['HouseholdPercent'] = df_no_outliers['HouseholdPercent'] ** (1/3)

df_no_outliers['PremLife'] = df_no_outliers['PremLife'] ** (1/3)
df_no_outliers['LifePercent'] = df_no_outliers['LifePercent'] ** (1/3)

df_no_outliers['PremWork'] = df_no_outliers['PremWork'] ** (1/3)
df_no_outliers['WorkPercent'] = df_no_outliers['WorkPercent'] ** (1/2)

In [ ]:
df_no_outliers[metric_features].skew(axis = 0)

| | Before | After |
| ---- | ---- | ---- |
|total_premiums| 1.501510 | 1.326973|
|PremHousehold   |   1.751577 | 0.302162|
|HouseholdPercent   |   0.717002 | -0.404910|
|PremLife   |   1.931087 | 0.325410|
|LifePercent   |   2.097619 | 0.231725|
|PremWork   |   1.920139 | 0.348968|
|WorkPercent   |   2.143754 |0.795744|


It is not possible to unskewe total_reversals

### Scaling <a class="anchor" id="scaling"></a>

We chose StandardScaler to scale the metric features because Standardization can help to improve the performance of k-means and other clustering algorithms by ensuring that the features are on a similar scale and have similar variances. Also, we have removed some skeweness from the data, making it more appropriate to use this scaler.

In [ ]:
df_discarded

In [ ]:
scaler = StandardScaler()

scaled_temp = scaler.fit_transform(df_no_outliers[metric_features])

df_scaled = pd.DataFrame(scaled_temp, columns = metric_features, index = df_no_outliers.index)

df_scaled = pd.concat([df_scaled, df_no_outliers[non_metric_features]], axis = 1, join = 'inner')

In [ ]:
# we scale the outliers using the same scaler that was fitted to the data that will be used for clustering
scaled_temp = scaler.transform(df_discarded[metric_features])

df_discarded_num = pd.DataFrame(scaled_temp, columns = metric_features, index = df_discarded.index)

df_discarded = pd.concat([df_discarded_num, df_discarded[non_metric_features]], axis = 1, join = 'inner')

In [ ]:
df_discarded.describe()

### Missing Values - Remaining Variables <a class="anchor" id="missing"></a>

In [ ]:
df_scaled.isna().sum()

**GeoLivArea**

In [ ]:
df_scaled[df_scaled["GeoLivArea"].isna()]

In [ ]:
df_discarded = pd.concat([df_discarded, df_scaled[df_scaled["GeoLivArea"].isna()]])

In [ ]:
# we chose to drop because it was only one observation and it has a lot of missing values in the other columns
df_scaled.drop(df_scaled[df_scaled["GeoLivArea"].isna()].index.values, axis=0, inplace=True)

**BirthYear**

For the BirthYear we just used the median beacuse we considered that none of the other variables would give us information to fill this missing values with the KNN imputer

In [ ]:
df_scaled['BirthYear'] = df_scaled['BirthYear'].fillna(df_scaled['BirthYear'].median())

**FirstPolYear**

For the *FirstPolYear* missing values we only used information about the *BirthYear* of the 5 nearest neighbors because it was the only variable that we considered relevant

In [ ]:
imputer = KNNImputer()
df_temp = imputer.fit_transform(df_scaled[['BirthYear','FirstPolYear']])
df_scaled['FirstPolYear'] = pd.DataFrame(df_temp, columns=['BirthYear','FirstPolYear'], index = df_scaled.index)['FirstPolYear']

**MonthSal**

We filled MonthSal missing values using information about *BirthYear* and *Premwork*


In [ ]:
imputer = KNNImputer()
df_temp = imputer.fit_transform(df_scaled[['BirthYear', 'MonthSal','PremWork']])
df_scaled['MonthSal'] = pd.DataFrame(df_temp, columns=['BirthYear', 'MonthSal','PremWork'], index = df_scaled.index)['MonthSal']

**Children**

We filled the missing values of column *Children* based only on *BirthYear* and *MonthSal* of the 5 nearest neighbors

In [ ]:
imputer = KNNImputer()

df_temp = imputer.fit_transform(df_scaled[['BirthYear','MonthSal','Children']])

df_scaled['Children'] = pd.DataFrame(df_temp, columns=['BirthYear','MonthSal','Children'], index = df_scaled.index)['Children']
df_scaled.isna().sum()

In [ ]:
# We are using 5 neighbors. If 3 of them have children, then 'Children' was converted to 3/5 = 0.6
# This means that Children >= 0.6 -> 1; Children < 0.6 -> 0
df_scaled['Children'].value_counts()

In [ ]:
df_scaled['Children'] = df_scaled['Children'].apply(lambda x: 1 if x >= 0.6 else 0)

In [ ]:
df_scaled['Children'].value_counts()

**EducDeg**

We filled EducDeg using information about *BirthYear* and *MonthSal* to calculate the nearest neighbors and then imputing with the mode of the K nearest neighbors

In [ ]:
class KNNModeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5, metric='euclidean', feat_to_impute = None, feats_nn = None):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.feats_nn = feats_nn
        self.feat_to_impute = feat_to_impute

    def fit(self, X, y=None):
        self.X_ = X
        self.is_fitted_ = True
        return self

    def transform(self, X):
      check_is_fitted(self, 'is_fitted_')

      # create kdtree
      tree = KDTree(self.X_[self.feats_nn], metric=self.metric)

      imputed_X = X.copy()

      missing_rows = X[X[self.feat_to_impute].isna()]

      distances, indices = tree.query(missing_rows[self.feats_nn].values, k= self.n_neighbors)

      missing_idx = np.array(missing_rows.index)

      for m_idx, n_idx in enumerate(indices):
        k_neighbors = self.X_.loc[n_idx, self.feat_to_impute]
        mode = Counter(k_neighbors).most_common(1)[0][0]
        imputed_X.loc[missing_idx[m_idx], self.feat_to_impute] = mode

      return imputed_X

In [ ]:
# Impute the missing values using KNN Mode
imputer = KNNModeImputer(n_neighbors = 5, feat_to_impute = 'EducDeg', feats_nn = ['BirthYear', 'MonthSal'])
df_imputed = imputer.fit_transform(df_scaled)

# The imputed dataset is returned as a NumPy array, so we need to convert it back to a Pandas dataframe
df_scaled = pd.DataFrame(df_imputed, columns=df_scaled.columns)

In [ ]:
df_scaled['EducDeg'].value_counts()

In [ ]:
df_scaled.isna().sum()

### Outliers using DBScan <a class="anchor" id="outliersDBScan"></a>

After filling the missing values, we can re-check the presence of outliers

In [ ]:
def plot_nearest_neighbors_dist(df, minNeigh):
  """
    Auxiliary function that plots the distance to the furthest neighbour among minNeigh nearest neighbors
  """

  neigh = NearestNeighbors(n_neighbors = minNeigh)
  neigh.fit(df)

  dists, _ = neigh.kneighbors(df)

  # Sorted distances to the minPts neighbors
  dists = np.sort(dists[:, -1])

  plt.figure(figsize = (12, 10))
  plt.xlabel('Distances')
  plt.ylabel('Relative frequency')
  plt.title('Plot to choose the best epsilon to use in DBScan')
  plt.plot(dists)

In [ ]:
# we do not use 'ClaimsRate' and 'CustMonVal' at the same, we just use one or the other because
# we know they are redundant as we have seen in exploration
# We do not use 'MonthSal' and 'BirthYear' for the same reason.
# We also do not use Prem'Percent' features because they are redundant with the original Prem features

metric_features_pre_engineering = ['FirstPolYear', 'MonthSal', 'ClaimsRate', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']

plot_nearest_neighbors_dist(df_scaled[metric_features_pre_engineering], 2*len(metric_features_pre_engineering))

In [ ]:
dbscan = DBSCAN(eps=1.7, min_samples=(2 * len(metric_features_pre_engineering)), n_jobs=4)
dbscan_labels = dbscan.fit_predict(df_scaled[metric_features_pre_engineering])

In [ ]:
df_scaled[dbscan_labels == -1]

In [ ]:
df_discarded = pd.concat([df_discarded, df_scaled[dbscan_labels == -1]])

In [ ]:
df_discarded.shape

In [ ]:
# we remove the outliers
df_scaled = df_scaled[dbscan_labels != -1]

In [ ]:
print(f'Percentage of total removed data: {np.round(((10296 - df_scaled.shape[0]) / 10296) * 100, 4)} %')

In [ ]:
df_scaled.shape

## Segmentation perspectives <a class="anchor" id="perpectives"></a>

### Socio-Demographic <a class="anchor" id="sociodemo"></a>

Feature we can use:

- BirthYear
- EducDeg
- MonthSal
- GeoLivArea
- Children

### Value <a class="anchor" id="value"></a>

- FirstPolYear
- MonthSal
- CustMonVal
- ClaimsRate
- total_premiums
- total_reversals

### Product <a class="anchor" id="product"></a>

For global interpretation comparing all clients with each other
- PremMotor
- PremHousehold
- PremHealth
- PremLife
- PremWork

For interpretation about individual behaviour of each client in comparison with the others
- MotorPercent
- HouseholdPercent
- HealthPercent
- LifePercent
- WorkPercent

## Assessment of the Discriminative Power of Features <a class="anchor" id="discriminativePower"></a>

### Numerical Variables <a class="anchor" id="numerical"></a>




#### Pairplot of metric features <a class="anchor" id="pairplot"></a>

In [ ]:
plot = sns.pairplot(df_scaled[metric_features])
plot.fig.suptitle('Pairplot of metric features', fontsize = 30)
plt.subplots_adjust(top=0.9)

#### Spearman Correlation between metric and ordinal features <a class="anchor" id="spearman"></a>

In [ ]:
fig = plt.figure(figsize=(20, 20))

corr = np.round(df_scaled[metric_features + ['EducDeg']].corr(method="spearman"), decimals=2)

annotation = np.where(np.absolute(corr.values) >= 0.5, corr.values, np.full(corr.shape,""))

sns.heatmap(data=corr, annot=annotation, cmap='BrBG', fmt='s', center=0, square=True, linewidths=.5)

fig.suptitle("Numerical + Ordinal features Spearman Correlation Matrix after preprocessing", fontsize = 20, y = 0.85, x = 0.45)
plt.show()

*ClaimsRate* and *CustMonVal* are highly correlated as we have seen before. *BirthYear* and *MonthSal* are also highly correlated, so they should not be used togheter. Regrading premiums, we should either use the total values or the percentage values, but never all togheter.


*ClaimsRate* and *CustMonVal* are also not really correlated with any other variable, thus we will not consider them for clustering

#### Component Planes <a class="anchor" id="componentPlanes"></a>

In [ ]:
np.random.seed(20)

# 15 x 15 SOM hexagonal matrix
sm = sompy.SOMFactory().build(
    df_scaled[metric_features].values, 
    mapsize=[15, 15],
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=metric_features
)
sm.train(n_job=4, verbose = False, train_rough_len=100, train_finetune_len=100)

In [ ]:
sns.set()
view2D = View2D(12, 12, "", text_size=10)
view2D.show(sm, col_sz = 4, cmap = 'Purples')
plt.suptitle("Component Planes", fontsize=20)
plt.show()

*total_reversals* does not seem to have good discriminative power, thus it will only be used for profilling and not for clustering.

### Categorical Variables <a class="anchor" id="categorical"></a>

In [ ]:
# Children vs Geolivarea

# Percentage of clients living in each area knowing they have and have not children
df_percent = df_scaled.groupby('Children')['GeoLivArea'].value_counts(normalize = True).reset_index(name = 'percentage')
sns.barplot(data = df_percent, x = 'Children', y = 'percentage', hue = 'GeoLivArea')

# Same percentage of people from all areas among people with and without children

In [ ]:
# Children vs Educdeg

df_percent = df_scaled.groupby('Children')['EducDeg'].value_counts(normalize = True).reset_index(name = 'percentage')
sns.barplot(data = df_percent, x = 'Children', y = 'percentage', hue = 'EducDeg')

# Same percentage of people with all education degrees among people with and without children

In [ ]:
# GeoLivArea vs EducDeg

sns.histplot(data = df_scaled, x = 'EducDeg', hue = 'GeoLivArea')

# Again, we see a really similar percentage of people from different areas in each EducDeg value

In [ ]:
# Children vs Birthyear

sns.histplot(data = df_scaled, x = 'BirthYear', hue = 'Children')

# The existence of children varies a lot with the age

In [ ]:
# Geolivarea vs Birthyear

sns.violinplot(data=df, x="GeoLivArea", y="BirthYear")

# No big of a difference between living areas among people from different ages

In [ ]:
# Educdeg vs Birthyear

sns.violinplot(data=df, x="EducDeg", y="BirthYear")

In [ ]:
# Birthyear vs Geolivarea vs Children

sns.violinplot(data=df, x="GeoLivArea", y="BirthYear", hue="Children", split=True)

# Really similar distributions

In [ ]:
# Birthyear vs Educdeg vs Children

sns.violinplot(data=df, x="EducDeg", y="BirthYear", hue="Children", split=True)

# This combination gives us different distributions of data, thus being ideal for clustering

We choose to exclude GeoLivArea from clustering as it does not seem to introduce any discriminative power when used togheter with the other Socio-Demographic variables.

Conclusion:

Socio-demographic clustering:
- BirthYear
- EducDeg
- Children

Value:

- FirstPolYear
- MonthSal
- total_premiums

Product

- PremMotor
- PremHousehold
- PremHealth
- PremLife
- PremWork

and

- MotorPercent
- HouseholdPercent
- HealthPercent
- LifePercent
- WorkPercent

## Determining the Clustering Tendency of Data <a class="anchor" id="tendency"></a>

After some analysis of the discriminative power of features and choosing the final ones, we decided to also analyse the clustering tendency of data using the sets of features selected.
This could help us avoid the time and effort of performing unnecessary clustering steps without knowing more confidently that the results would be satisfactory

### 3D visualizations <a class="anchor" id="3DVisualizations"></a>

**Value**

In [ ]:
value_feats = ['FirstPolYear', 'MonthSal', 'total_premiums']

In [ ]:
fig = px.scatter_3d(df_scaled, x='FirstPolYear', y='MonthSal', z='total_premiums')

fig.update_traces(marker_size = 2)

fig.update_layout(
    title_text='3D Scatter Plot of Value features',
    title_font_size = 18,
    height=500,
    width=700,
    scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1))
)

fig.show()

**Product**

In [ ]:
product_feats = ["PremMotor", "PremHousehold", "PremHealth", "PremLife", "PremWork"]
product_feats2 = ["MotorPercent", "HouseholdPercent", "HealthPercent","LifePercent","WorkPercent"]

In [ ]:
combs = list(itertools.combinations(product_feats,3))

n_cols = 5
n_rows = 2

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    specs=[[{'type': 'scene'}]*n_cols]*n_rows,
    subplot_titles=[f"{comb[0]}, {comb[1]}, {comb[2]}" for comb in combs])

for i, comb in enumerate(combs):
  r = floor(i / n_cols) + 1
  c = (i % n_cols) + 1

  fig.add_scatter3d(
    x=df_scaled[comb[0]], y=df_scaled[comb[1]], z=df_scaled[comb[2]],
    row=r, col=c, mode='markers', showlegend=False)
  
fig.update_layout(
    title_text='3D Scatter Plots of the combinations of the first set of Product features',
    title_font_size = 18,
    height=800,
    width=1600
)

fig.update_traces(marker_size=2)

fig.update_annotations(font_size=12)
fig.show()


In [ ]:
combs = list(itertools.combinations(product_feats2,3))

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    specs=[[{'type': 'scene'}]*n_cols]*n_rows,
    subplot_titles=[f"{comb[0]}, {comb[1]}, {comb[2]}" for comb in combs])

for i, comb in enumerate(combs):
  r = floor(i / n_cols) + 1
  c = (i % n_cols) + 1

  fig.add_scatter3d(
    x=df_scaled[comb[0]], y=df_scaled[comb[1]], z=df_scaled[comb[2]],
    row=r, col=c, marker = go.scatter3d.Marker(size=2), mode='markers', showlegend=False)
  
fig.update_layout(
    title_text='3D Scatter Plots of the combinations of the second set of Product features',
    title_font_size = 18,
    height=800,
    width=1600
)

fig.update_annotations(font_size=12)
fig.show()


### Distance matrix <a class="anchor" id="distanceMatrix"></a>

**Socio-demographic**

In [ ]:
distance_matrix = gower.gower_matrix(df_scaled[['BirthYear', 'EducDeg', 'Children']], cat_features = [False, True, True])

fig = plt.figure(figsize=(30, 30))
plt.imshow(distance_matrix, cmap = 'plasma')
plt.title('Pairwise distance matrix under a Socio-Demographic clustering perspective', fontsize = 50)
plt.show()

**Value**

In [ ]:
def plot_distance_matrix(df, perspective):

  plt.figure(figsize = (60, 60))
  distance_df = squareform(pdist(df))

  fig = plt.figure(figsize=(30, 30))
  plt.imshow(distance_df, cmap = 'plasma')
  plt.title(f'Pairwise distance matrix under a {perspective} clustering perspective', fontsize = 50)
  plt.show()

In [ ]:
plot_distance_matrix(df_scaled[value_feats], 'Value')

**Product**

In [ ]:
plot_distance_matrix(df_scaled[product_feats], 'Product (total premiums)')

In [ ]:
plot_distance_matrix(df_scaled[product_feats2], 'Product (premiums percentage)')

### Hopkins test <a class="anchor" id="hopkinsTest"></a>

Tests if data is uniformly distributed

In [ ]:
from random import sample
from numpy.random import uniform

# function to compute hopkins's statistic for the dataframe X
def hopkins_statistic(X):
    
    X=X.values  #convert dataframe to a numpy array
    sample_size = int(X.shape[0]*0.05) #0.05 (5%) based on paper by Lawson and Jures
    
    #a uniform random sample in the original data space
    X_uniform_random_sample = uniform(X.min(axis=0), X.max(axis=0) ,(sample_size , X.shape[1]))
    
    #a random sample of size sample_size from the original data X
    random_indices=sample(range(0, X.shape[0], 1), sample_size)
    X_sample = X[random_indices]
   
    #initialise unsupervised learner for implementing neighbor searches
    neigh = NearestNeighbors(n_neighbors=2)
    nbrs=neigh.fit(X)
    
    #u_distances = nearest neighbour distances from uniform random sample
    u_distances , u_indices = nbrs.kneighbors(X_uniform_random_sample , n_neighbors=2)
    u_distances = u_distances[: , 0] #distance to the first (nearest) neighbour
    
    #w_distances = nearest neighbour distances from a sample of points from original data X
    w_distances , w_indices = nbrs.kneighbors(X_sample , n_neighbors=2)
    #distance to the second nearest neighbour (as the first neighbour will be the point itself, with distance = 0)
    w_distances = w_distances[: , 1]
    
    u_sum = np.sum(u_distances)
    w_sum = np.sum(w_distances)
    
    #compute and return hopkins' statistic
    H = u_sum/ (u_sum + w_sum)
    return H

# SOURCE: https://github.com/prathmachowksey/Hopkins-Statistic-Clustering-Tendency/blob/master/Hopkins-Statistic-Clustering-Tendency.ipynb

**Value**

In [ ]:
l = [] #list to hold values for each call
for i in range(20):
    H=hopkins_statistic(df_scaled[value_feats])
    l.append(H)

np.mean(l) # really high tendency!

**Product**

In [ ]:
l = []
for i in range(20):
    H=hopkins_statistic(df_scaled[product_feats])
    l.append(H)

np.mean(l) # really high tendency!

In [ ]:
l = []
for i in range(20):
    H=hopkins_statistic(df_scaled[product_feats2])
    l.append(H)

np.mean(l) # really high tendency!

## Clustering <a class="anchor" id="clustering"></a>

For ***Socio-Demographic***:

K-Prototypes:
1. Find best number of cluster using the elbow method

Hierarchical Clustering
1. Calculate gower distance matrix
2. Choose best linkage by plotting silhouette scores across a number of clusters
3. Choose best number of clusters with a dendogram

K-Medoids (PAM)
1. Choose best number of clusters using elbow and silhouette

<br>

For both ***Value*** and ***Product***:

Hierarquical:
1. For each linkage, plot the r2 scores across a number of clusters to determine the best one
2. Choose the ideal number of clusters using the dendogram

Kmeans:
To choose the best number of clusters
1. Elbow method
2. Silhouette score
3. K-Means with large nr of cluster followed by HC

Mean-Shift:
1. Estimate bandwidth

DBScan
1. Define min_samples
2. Define eps

SOM
1. Train SOM
2. Plot U-Matrix
3. Plot HitMap

SOM + hierarquical:
1. Choose best linkage by plotting r2 scores for clustering the SOM units across a number of clusters
2. Choose best number of cluster using the dendogram

SOM + kmeans:
1. Same as K-Means but this time clustering the SOM units instead of the original data points

For each set of features: Perform the clustering and get r2 score and silhouette



#### Auxiliary functions

In [ ]:
# ----------------------- GENERAL Auxiliary Functions ---------------------------------#
def get_sst(df):
  """
    Calculates the um of squared distances of each point to the mean of the points

  """
  return np.sum(df.var() * (df.count() - 1))

def get_r2(df, labels):
  """
    Calculates r2 given a dataframe and the clustering solution
  """
  sst = get_sst(df)
  ssw = np.sum(df.groupby(labels).apply(get_sst))
  ssb = sst - ssw
  return ssb / sst

# ----------------------- HIERARQUICAL Auxiliary Functions ---------------------------------#

def get_hierarquical_scores(df, min_clust, max_clust):
  """
    Returns r2 scores for each nr of clusters and each linkage method
  """
  hc_r2_scores = {}

  # get r2 scores for each model
  for linkage in ['single', 'complete', 'average', 'ward']:
    r2_linkage = {}
    # get the r2 scores for each number of clusters
    for n_clust in range(min_clust, max_clust + 1):
      model = AgglomerativeClustering(linkage = linkage, n_clusters = n_clust)
      labels = model.fit_predict(df)
      r2_linkage[n_clust] = get_r2(df, labels)

    hc_r2_scores[linkage] = r2_linkage

  return hc_r2_scores

def choose_hc_linkage(df, min_clust, max_clust, description):
  """
    Plots the r2 scores for each nr of clusters and each linkage method
  """
  r2_scores = get_hierarquical_scores(df, min_clust, max_clust)

  r2_df = pd.DataFrame(r2_scores)

  sns.set()
  fig = plt.figure(figsize = (10, 5))
  sns.lineplot(data = r2_df, linewidth = 2.5, markers = ['o'] * 4)

  fig.suptitle(f"R2 plot for various linkages and nr of clusters for {description}", fontsize = 15)
  plt.gca().invert_xaxis()
  plt.legend(title = 'Clustering Algorithm', title_fontsize = 12)
  plt.xticks(range(1, max_clust + 1))
  plt.xlabel("Number of clusters", fontsize = 10)
  plt.ylabel("R2", fontsize = 10)

  plt.show()

def plot_dendogram(hclust, y_threshold, purpose, metric):
  """
    Creates linkage matrix and then plots the dendrogram
  """
  # Adapted from:
  # https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py
  linkage = hclust.get_params()['linkage']
  distance = hclust.get_params()['metric']
  plt.figure(figsize = (10, 10))

  # create the counts of samples under each node
  counts = np.zeros(hclust.children_.shape[0])
  n_samples = len(hclust.labels_)
  for i, merge in enumerate(hclust.children_):
      current_count = 0
      for child_idx in merge:
          if child_idx < n_samples:
              current_count += 1  # leaf node
          else:
              current_count += counts[child_idx - n_samples]
      counts[i] = current_count

  linkage_matrix = np.column_stack([hclust.children_, hclust.distances_, counts]).astype(float)
  
  # Plot the corresponding dendrogram
  dendrogram(linkage_matrix, color_threshold=y_threshold)
  plt.title(purpose)
  plt.xlabel("Number of points in node (or index of point if no parenthesis).")
  plt.ylabel(f"{metric} distance")
  plt.show()

# ----------------------- K-MEANS auxiliary functions ---------------------------------#
def get_kmeans_inertia_silhouette(df, min_clust, max_clust):
  """ 
    Returns values of inertia and silhouette for a range of number of clusters
  """
  inertia = []
  silhouette = []
  for n_clus in range(min_clust, max_clust):  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters = n_clus, init = 'k-means++', n_init = 15, random_state = 1)
    kmclust.fit(df)
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

    # skip n_clus == 1 because it only makes sense to compute silhouette with at least 2 clusters
    if n_clus == 1:
      continue
    labels = kmclust.predict(df)
    silhouette.append(silhouette_score(df, labels)) # save silhouette of the given solution

  return inertia, silhouette

# ----------------------- K-MEANS and K-PROTOTYPES auxiliary functions ---------------------------------#

def plot_scores(inertia, silhouette, description, clust_range):
  f = plt.figure(figsize=(20,5))

  ax1 = f.add_subplot(121)
  ax1.plot(range(clust_range[0], clust_range[1]), inertia)
  ax1.set_ylabel("Inertia = SSw")
  ax1.set_xlabel('Nr clusters')
  ax1.set_title(f'Elbow method for {description}', fontsize = 20)

  ax2 = f.add_subplot(122)
  ax2.plot(range(clust_range[0] + 1, clust_range[1]), silhouette)
  ax2.set_ylabel("Avg silhouette")
  ax2.set_xlabel('Nr clusters')
  ax2.set_title(f'Avg silhouette for {description}', fontsize = 20)


# ----------------- SOM auxiliary function ----------------- #

def create_som(df, mapsize):
  """
    Instantiates and trains a SOM
  """

  np.random.seed(20)

  sm = sompy.SOMFactory().build(
      df.values, 
      mapsize=mapsize,
      initialization='random', 
      neighborhood='gaussian',
      training='batch',
      lattice='hexa',
      component_names=df.columns
  )
  sm.train(n_job=4, verbose = False, train_rough_len=100, train_finetune_len=100)

  return sm

def plot_umat(description, sm):
  """
    U-Matrix plot: distances between neurons
  """
  u = sompy.umatrix.UMatrixView(9, 9, f'{description} U-Matrix', show_axis=True, text_size=20, show_text=True)

  UMAT = u.show(
      sm, 
      distance=2,
      row_normalized=False,
      show_data=True, 
      contour=True,
      blob=True
  )

def plot_hitmap(description, sm):
  """
    Hit-Map plot: number of observations that consider a neuron its BMU
  """

  vhts  = BmuHitsView(12,12,f"{description} Hit Map")
  vhts.show(sm, anotate=True, onlyzeros=False, labelsize=12, cmap="GnBu")
  plt.show()

def cluster_som_units(model, sm, description):
  """
    Runs the clustering algorithm (model) on SOM units
  """
  nodeclus_labels = model.fit_predict(sm.codebook.matrix)
  sm.cluster_labels = nodeclus_labels

  hits = HitMapView(12, 12, description, text_size = 15)
  hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap="Pastel1")
  plt.show()
  return sm

def kmeans_on_som_plot(n_clusters, sm, description):
  kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=20, random_state=1)

  return cluster_som_units(kmeans, sm, description)

def hc_on_som_plot(n_clusters, sm, description, linkage):
  hc = AgglomerativeClustering(n_clusters=n_clusters, linkage = linkage)
  
  return cluster_som_units(hc, sm, description)

def merge_bmu_labels(df, sm):
  """
    Returns a match between the BMU label and the observations that have that BMU
  """

  df_nodes = pd.DataFrame(sm.codebook.matrix, columns=df.columns)
  df_nodes['label'] = sm.cluster_labels
  
  bmus_map = sm.find_bmu(df)[0]  # Get BMU for each observation in df

  df_bmus = pd.DataFrame(
      np.concatenate((df, np.expand_dims(bmus_map,1)), axis=1),
      index=df.index, columns=np.append(df.columns,"BMU")
  )
  
  # Labels for each observation based on BMU label
  df_final = df_bmus.merge(df_nodes['label'], 'left', left_on="BMU", right_index=True)

  return df_final

### Socio-Demographic <a class="anchor" id="sociodemoClust"></a>

In [ ]:
sociodemo_results = pd.DataFrame(columns = ['silhouette']) # To store the silhouette scores of each algorithm

In [ ]:
df_demo = df_scaled[['BirthYear', 'EducDeg', 'Children']].copy()

#### K-Prototypes <a class="anchor" id="kprototypes"></a>

__1.__ Choose best K using the elbow method

In [ ]:
cost = []
for cluster in range(2, 10):
  kprototype = KPrototypes(n_jobs = 4, n_clusters = cluster, random_state = 0)
  kprototype.fit_predict(df_demo, categorical = [1, 2])
  cost.append(kprototype.cost_)
  print('Cluster initiation: {}'.format(cluster))

In [ ]:
plt.figure(figsize = (15, 12))
plt.plot(range(2, len(cost) + 2), cost)
plt.suptitle('Cost curve of K-Prototypes', fontsize = 20)
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

Elbow is approximately at 6 clusters

In [ ]:
kprototype = KPrototypes(n_jobs = 4, n_clusters = 6, random_state = 0)
kprototype.fit_predict(df_demo, categorical = [1, 2])

In [ ]:
kprototype_scores = {}
labels = kprototype.labels_
from sklearn.metrics.pairwise import euclidean_distances

def kprototype_distance(df, numerical_vars, categorical_vars):
  
  # euclidean distance for numerical feature
  def pairwise_distance(x1, x2):
    return np.abs(x1 - x2)

  # apply the function to the observations series to calculate the pairwise distances
  dist = df[numerical_vars].apply(lambda x: pairwise_distance(x, df[numerical_vars]))

  # pairwise hamming distance between all rows in the categorical features
  dist = dist + squareform(pdist(df[categorical_vars], 'hamming'))

  return dist

dist_matrix = kprototype_distance(df_demo, 'BirthYear', ['EducDeg', 'Children'])

In [ ]:
kprototype_scores['k-prototypes'] = [silhouette_score(dist_matrix, labels)]
kprototype_scores_df = pd.DataFrame(kprototype_scores, index=['silhouette']).T
kprototype_scores_df.dropna(axis=1, how='all', inplace=True)
sociodemo_results = pd.concat([sociodemo_results, kprototype_scores_df])

#### Hierarchical Clustering <a class="anchor" id="hcSociodemo"></a>

We can not use ward linkage with gower distance, so we will only search among the

__1.__ Calculate distance matrix using gower distance

In [ ]:
distance_matrix = gower.gower_matrix(df_demo, cat_features = [False, True, True])

__2.__ Choose best linkage by plotting silhouette scores across a number of clusters


In [ ]:
silhouette_scores = {}

# get silhouette scores for each model
for linkage in ['single', 'complete', 'average']:
  silhouette_linkage = {}
  # get the silhouette scores for each number of clusters
  for n_clust in range(2, 10):
    model = AgglomerativeClustering(linkage = linkage, n_clusters = n_clust, metric='precomputed')
    labels = model.fit_predict(distance_matrix)
    silhouette_linkage[n_clust] = silhouette_score(distance_matrix, labels)

  silhouette_scores[linkage] = silhouette_linkage

silhouette_df = pd.DataFrame(silhouette_scores)

In [ ]:
sns.set()
fig = plt.figure(figsize = (10, 5))
sns.lineplot(data = silhouette_df, linewidth = 1.5, markers = ['o'] * 3)

fig.suptitle(f"Silhouette plot for various linkage methods and number of clusters for Socio-Demographic perspective ", fontsize = 15)
plt.gca().invert_xaxis()
plt.legend(title = 'Clustering Algorithm', title_fontsize = 12)
plt.xticks(range(2, 10))
plt.xlabel("Number of clusters", fontsize = 10)
plt.ylabel("Silhouette", fontsize = 10)

plt.show()

___3.___ Choose best number of clusters with a dendogram


In [ ]:
clst = AgglomerativeClustering(linkage='average', distance_threshold=0, n_clusters=None, metric='precomputed')
clst.fit_predict(distance_matrix)
plot_dendogram(clst, 0.45, 'clustering following a Socio-Demographic perspective', 'precomputed')

In [ ]:
hc_scores = {}

clst = AgglomerativeClustering(linkage='average', n_clusters=3, metric='precomputed')
labels = clst.fit_predict(distance_matrix)
hc_scores['hc'] = [silhouette_score(distance_matrix, labels)]

In [ ]:
hc_scores_df = pd.DataFrame(hc_scores, index=['silhouette']).T
sociodemo_results = pd.concat([sociodemo_results, hc_scores_df])
sociodemo_results

#### K-Medoids (PAM) <a class="anchor" id="kmedoids"></a>

__1.__ Choose best number of clusters using elbow and silhouette


In [ ]:
inertia = []
silhouette = []
for n_clus in range(1, 10):
  km = kmedoids.KMedoids(n_clus, method='fasterpam', random_state = 0).fit(distance_matrix)
  inertia.append(km.inertia_)

  # we can not calculate the silhouette for a solution with only one cluster, so we skip it
  if n_clus == 1:
    continue

  silhouette.append(silhouette_score(distance_matrix, km.labels_))

plot_scores(inertia, silhouette, 'K-Medoids (Socio-Demographic)', (1, 10))

In [ ]:
kmedoids_scores = {}

km = kmedoids.KMedoids(5, method='fasterpam').fit(distance_matrix)
labels = km.labels_
kmedoids_scores['k-medoids'] = [silhouette_score(distance_matrix, labels)]

In [ ]:
kmedoids_scores_df = pd.DataFrame(kmedoids_scores, index=['silhouette']).T
sociodemo_results = pd.concat([sociodemo_results, kmedoids_scores_df])
sociodemo_results

### Value <a class="anchor" id="valueClust"></a>

In [ ]:
value_results = pd.DataFrame(columns = ['r2', 'silhouette'])

In [ ]:
df_value = df_scaled[['FirstPolYear', 'MonthSal', 'total_premiums']].copy()

#### Cell-based segments <a class="anchor" id="cellbased"></a>

Using *ClaimsRate* and *total_premiums*

Costumers divided into 4 segments:
* Profitable: Low (q1 and q2) ClaimsRate and High (q3 and q4) total_premiums
* Risk-Averse: Low (q1 and q2) ClaimsRate and Low (q1 and q2) total_premiums
* Risk-Loving: High (q3 and q4)  ClaimsRate and High (q3 and q4)  total_premiums 
* Harmful: High (q3 and q4) ClaimsRate and Low (q1 and q2) total_premiums




In [ ]:
df_quartiles = df_scaled.copy()
quartiles = ['q1', 'q2', 'q3', 'q4']
# partition the customers into ClaimsRate quartils
df_quartiles['ClaimsRate_quartil'] = pd.qcut(df_scaled['ClaimsRate'], 4, quartiles)
# partition the customers into total_premiums quartils
df_quartiles['total_premiums_quartil'] = pd.qcut(df_scaled['total_premiums'], 4, quartiles)

quartil_matrix = pd.crosstab(df_quartiles['ClaimsRate_quartil'], df_quartiles['total_premiums_quartil'])

sns.set()
plt.figure(figsize=(10, 10))
plt.suptitle('Contingency Table', fontsize = 15, y = 0.9)
plt.xlabel('total_premiums Quartiles')
plt.ylabel('ClaimsRate Quartiles')
sns.heatmap(quartil_matrix, annot = True, fmt='.3g', square = True)

In [ ]:
df_quartiles

#### Hierarchical Clustering <a class="anchor" id="hcValue"></a>

__1.__ For each linkage, plot the r2 scores across a number of clusters to determine the best one


In [ ]:
choose_hc_linkage(df_value, 1, 5, 'clustering follwoing a Value perspective')

The best linkage is *ward*

__2.__ Choose the ideal number of clusters using the dendogram

In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
clst.fit_predict(df_value)
plot_dendogram(clst, 70, 'clustering following a Value perspective', 'precomputed')

In [ ]:
hc_scores = {}

clst = AgglomerativeClustering(linkage='ward', n_clusters = 4)
labels = clst.fit_predict(df_value)
hc_scores['hc'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

In [ ]:
hc_scores_df = pd.DataFrame(hc_scores, index = ['r2', 'silhouette']).T
hc_scores_df = hc_scores_df.dropna(axis=1, how='all')
value_results = pd.concat([value_results, hc_scores_df])
value_results

#### K-Means <a class="anchor" id="kmeansValue"></a>

__1./2.__ Elbow method and Silhouette score

In [ ]:
inertia, silhouette = get_kmeans_inertia_silhouette(df_value, min_clust = 1, max_clust = 10)
plot_scores(inertia, silhouette, 'K-Means following a value perspective', (1, 10))

__3.__ K-Means with large nr of cluster followed by HC

In [ ]:
# perform kmeans with a lot of clusters
kmclust = KMeans(n_clusters = 100, init = 'k-means++', n_init = 15, random_state = 1)
kmclust.fit(df_value)

# new 'observations' are the clusters centroids
centroids = pd.DataFrame(kmclust.cluster_centers_)

# choose best linkage to cluster the centroids
choose_hc_linkage(centroids, 1, 7, 'a Value perspective (HC on top of K-Means')

In [ ]:
# choose the best nr of clusters given the best linkage
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
clst.fit_predict(centroids)

plot_dendogram(clst, 8, 'deciding the nr of clusters of K-Means following a Value perspective', 'precomputed')

In [ ]:
kmeans_scores = {}

kmclust = KMeans(n_clusters = 4, init = 'k-means++', n_init = 15, random_state = 1)
labels = kmclust.fit_predict(df_value)
kmeans_scores['kmeans'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

In [ ]:
value_results = pd.concat([value_results, pd.DataFrame(kmeans_scores, index=['r2', 'silhouette']).T])
value_results

#### Mean-shift <a class="anchor" id="meanshiftValue"></a>

In [ ]:
mean_shift_scores = {}

ms = MeanShift(bin_seeding=True, n_jobs=4)
labels = ms.fit_predict(df_value)
if len(np.unique(labels)) == 1:
  print("Only one cluster found")
else:
  mean_shift_scores['df_value'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

#### DBScan <a class="anchor" id="dbscanValue"></a>

__1./2.__ Define *min_samples* and eps

In [ ]:
minPts = 2 * len(df_value.columns)
plot_nearest_neighbors_dist(df_value, minPts)

In [ ]:
dbscan_scores = {}
dbscan = DBSCAN(eps = 0.35, min_samples=minPts, n_jobs=4)
labels = dbscan.fit_predict(df_value)
print(f"nr of clusters found: {len(np.unique(labels))}")
dbscan_scores['dbscan'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

In [ ]:
value_results = pd.concat([value_results, pd.DataFrame(dbscan_scores, index=['r2', 'silhouette']).T])
value_results

#### SOM <a class="anchor" id="somValue"></a>

__1.__ Train SOM

In [ ]:
sm = create_som(df_value, [15, 15])

__2.__ Plot U-Matrix

__3.__ Plot Hitmap

In [ ]:
plot_hitmap('Value segmentation -', sm)

#### SOM + Hierarchical Clustering <a class="anchor" id="somhcValue"></a>

__1.__ Choose best linkage by plotting r2 scores for clustering the SOM units across a number of clusters


In [ ]:
units = pd.DataFrame(sm.codebook.matrix)
units.name = 'SOM_value'
choose_hc_linkage(units, 1, 5, 'deciding the linkage of HC on top of SOM following a Value perspective')

__2.__ Choose best number of cluster using the dendogram


In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
units = pd.DataFrame(sm.codebook.matrix)
clst.fit_predict(units)
plot_dendogram(clst, 9, 'deciding the nr of clusters of HC on top of SOM following a Value perspective', 'precomputed')

__3.__ HC on SOM units

In [ ]:
hc_som = hc_on_som_plot(4, sm, 'Hierarchical clustering on top of SOM following a Value perspective', 'ward')

__4.__ Merge BMU labels with observations

In [ ]:
final_df = merge_bmu_labels(df_value, hc_som)
final_df

In [ ]:
som_hc_scores = {}

labels = final_df['label']
som_hc_scores['som_hc'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

In [ ]:
value_results = pd.concat([value_results, pd.DataFrame(som_hc_scores, index=['r2', 'silhouette']).T])
value_results

#### SOM + K-Means <a class="anchor" id="somkmeansValue"></a>

__1./2.__ Elbow method and Silhouette score

In [ ]:
weights = sm.codebook.matrix

inertia, silhouette = get_kmeans_inertia_silhouette(weights, min_clust = 1, max_clust = 10)
plot_scores(inertia, silhouette, 'K-Means + SOM following a Value perspective', (1, 10))

__3.__ K-Means on SOM units


In [ ]:
kmeans_som = kmeans_on_som_plot(4, sm, 'K-Means on top of SOM following a Value perspective')

__4.__ Merge BMU labels with observations

In [ ]:
final_df = merge_bmu_labels(df_value, kmeans_som)

In [ ]:
kmeans_som_scores = {}

labels = final_df['label']
kmeans_som_scores['som_kmeans'] = [get_r2(df_value, labels), silhouette_score(df_value, labels)]

In [ ]:
value_results = pd.concat([value_results, pd.DataFrame(kmeans_som_scores, index=['r2', 'silhouette']).T])
value_results

### Product <a class="anchor" id="productClust"></a>

In [ ]:
product_results = pd.DataFrame(columns = ['r2', 'silhouette'])

In [ ]:
df_product_1 = df_scaled[['PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork']].copy()
df_product_1.name = 'df_product_premiums'

df_product_2 = df_scaled[["MotorPercent", "HouseholdPercent", "HealthPercent","LifePercent","WorkPercent"]].copy()
df_product_2.name = 'df_product_percent'

In [ ]:
df_product = [df_product_1, df_product_2]

#### Hierarchical Clustering <a class="anchor" id="hcProduct"></a>

__1.__ For each linkage, plot the r2 scores across a number of clusters to determine the best one


In [ ]:
sets = ['total premiums', 'premiums percentage']

for i, df in enumerate(df_product):
  choose_hc_linkage(df, 1, 5, f'Product perspective ({sets[i]})')

The best linkage is *ward* for all datasets

__2.__ Choose the ideal number of clusters using the dendogram

Total premiums

In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
clst.fit_predict(df_product_1)
plot_dendogram(clst, 80, 'clustering following a Product perspective (total premiums)', 'precomputed')

Premiums Percentage

In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
clst.fit_predict(df_product_2)
plot_dendogram(clst, 80, 'clustering following a Product perspective (premiums percentage)', 'precomputed')

In [ ]:
nr_clusters = [4, 3]
hc_scores = {}

for i, df in enumerate(df_product):
  clst = AgglomerativeClustering(linkage='ward', n_clusters=nr_clusters[i])
  labels = clst.fit_predict(df)
  hc_scores[df.name + 'hc'] = [get_r2(df, labels), silhouette_score(df, labels)]

In [ ]:
product_results = pd.concat([product_results, pd.DataFrame(hc_scores, index=['r2', 'silhouette']).T])
product_results

#### K-Means <a class="anchor" id="kmeansProduct"></a>

__1./2.__ Elbow method and Silhouette score

In [ ]:
for i, df in enumerate(df_product):
  inertia, silhouette = get_kmeans_inertia_silhouette(df, min_clust = 1, max_clust = 10)
  plot_scores(inertia, silhouette, f'{sets[i]}', (1, 10))

__3.__ K-Means with large nr of cluster followed by HC

In [ ]:
centroids_df = []
for i, df in enumerate(df_product):
  # perform kmeans with a lot of clusters
  kmclust = KMeans(n_clusters = 100, init = 'k-means++', n_init = 15, random_state = 1)
  kmclust.fit(df)

  # new 'observations' are the clusters centroids
  centroids = pd.DataFrame(kmclust.cluster_centers_)
  centroids.name = df.name + '_centroids'

  # save the dataframe for later
  centroids_df.append(centroids)

  # choose best linkage to cluster the centroids
  choose_hc_linkage(centroids, 1, 5, f'Product perspective ({sets[i]})')

Total premiums

In [ ]:
# choose the best nr of clusters given the best linkage
clst = AgglomerativeClustering(linkage='complete', distance_threshold=0, n_clusters=None)
clst.fit_predict(centroids_df[0])
plot_dendogram(clst, 5, 'deciding nr of clusters of K-Means following a Product perspective (total premiums set)','precomputed')

Premiums Percentage

In [ ]:
# choose the best nr of clusters given the best linkage
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
clst.fit_predict(centroids_df[1])
plot_dendogram(clst, 10, 'deciding nr of clusters of K-Means following a Product perspective (premiums percentage set)', 'precomputed')

In [ ]:
nr_clusters = [3, 3]
kmeans_scores = {}

for i, df in enumerate(df_product):
  kmclust = KMeans(n_clusters = nr_clusters[i], init = 'k-means++', n_init = 15, random_state = 1)
  labels = kmclust.fit_predict(df)
  kmeans_scores[df.name + '_K-Means'] = [get_r2(df, labels), silhouette_score(df, labels)]

In [ ]:
product_results = pd.concat([product_results, pd.DataFrame(kmeans_scores, index=['r2', 'silhouette']).T])
product_results

#### Mean-shift <a class="anchor" id="meanshiftProduct"></a>

In [ ]:
mean_shift_scores = {}

for df in df_product:
  ms = MeanShift(bin_seeding=True, n_jobs=4)
  labels = ms.fit_predict(df)
  if len(np.unique(labels)) == 1:
    print(f'Only one cluster found using {df.name}')
    continue
  mean_shift_scores[df.name] = [get_r2(df, labels), silhouette_score(df, labels)]

#### DBScan <a class="anchor" id="dbscanProduct"></a>

__1./2.__ Define *min_samples* and eps

In [ ]:
minPts = []
for df in df_product:
  ms = (2 * len(df.columns))
  minPts.append(ms)
  plot_nearest_neighbors_dist(df, ms)

In [ ]:
eps = [0.7, 0.75]

In [ ]:
dbscan_scores = {}
for i, df in enumerate(df_product):
  dbscan = DBSCAN(eps=eps[i], min_samples=minPts[i], n_jobs=4)
  labels = dbscan.fit_predict(df)
  print(f"Number of clusters found for {df.name}: {len(np.unique(labels))}")
  dbscan_scores[df.name + '_dbscan'] = [get_r2(df, labels), silhouette_score(df, labels)]

In [ ]:
product_results = pd.concat([product_results, pd.DataFrame(dbscan_scores, index=['r2', 'silhouette']).T])
product_results

#### SOM <a class="anchor" id="somProduct"></a>

__1./2.__ Train SOM and Plot U-Matrix

In [ ]:
soms = []

for i, df in enumerate(df_product):
  sm = create_som(df, [15, 15])
  soms.append(sm)

__3.__ Plot Hitmap

In [ ]:
for i, df in enumerate(df_product):
  plot_hitmap(f'Product segmentation using {sets[i]} set of features -', soms[i])

#### SOM + Hierarchical Clustering <a class="anchor" id="somhcProduct"></a>

__1.__ Choose best linkage by plotting r2 scores for clustering the SOM units across a number of clusters


In [ ]:
for i, sm in enumerate(soms):
  units = pd.DataFrame(sm.codebook.matrix)
  choose_hc_linkage(units, 1, 5, f'Product perspective ({sets[i]})')

__2.__ Choose best number of cluster using the dendogram


SOM Total Premiums

In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
units = pd.DataFrame(soms[0].codebook.matrix) # get weights of each neuron
clst.fit_predict(units) # employ HC on top of SOM units
plot_dendogram(clst, 10, 'HC + SOM following a Product perspective (total premiums set)', 'precomputed')

SOM Premiums Percentage


In [ ]:
clst = AgglomerativeClustering(linkage='ward', distance_threshold=0, n_clusters=None)
units = pd.DataFrame(soms[1].codebook.matrix)
clst.fit_predict(units)
plot_dendogram(clst, 11, 'HC + SOM following a Product perspective (premiums percentage set)', 'precomputed')

__3.__ HC on SOM units

In [ ]:
nr_clusters = [3, 3]
hc_som =[]
for i, df in enumerate(df_product):
  sm = soms[i]
  hc_som.append(hc_on_som_plot(nr_clusters[i], sm, f'HC on top of SOM for Product ({sets[i]})', 'ward'))

__4.__ Merge BMU labels with observations

In [ ]:
final_df = []

for i, df in enumerate(df_product):
  final_df.append(merge_bmu_labels(df, hc_som[i]))

In [ ]:
som_hc_scores = {}

for i, df in enumerate(final_df):
  labels = df['label']
  som_hc_scores[sets[i] + '_SOM_HC'] = [get_r2(df, labels), silhouette_score(df, labels)]

In [ ]:
product_results = pd.concat([product_results, pd.DataFrame(som_hc_scores, index=['r2', 'silhouette']).T])
product_results

#### SOM + K-Means <a class="anchor" id="somkmeansProduct"></a>

__1./2.__ Elbow method and Silhouette score

In [ ]:
for i, df in enumerate(df_product):
  sm = soms[i]

  weights = sm.codebook.matrix

  inertia, silhouette = get_kmeans_inertia_silhouette(weights, min_clust = 1, max_clust = 10)
  plot_scores(inertia, silhouette,  f'K-Means + SOM following a Product ({sets[i]}) perspective', (1, 10))

In [ ]:
nr_clusters = [3, 3]

__3.__ K-Means on SOM units


In [ ]:
kmeans_soms = []

for i, sm in enumerate(soms):
  kmeans_soms.append(kmeans_on_som_plot(nr_clusters[i], sm, f'K-Means on top of som for Product ({sets[i]})'))

__4.__ Merge BMU labels with observations

In [ ]:
final_dfs = []

for i, df in enumerate(df_product):
  final_dfs.append(merge_bmu_labels(df, kmeans_soms[i]))

In [ ]:
kmeans_som_scores = {}
for i, df in enumerate(df_product):
  labels = final_dfs[i]['label']
  kmeans_som_scores[sets[i] + '_SOM_KM'] = [get_r2(df, labels), silhouette_score(df, labels)]

In [ ]:
product_results = pd.concat([product_results, pd.DataFrame(som_hc_scores, index=['r2', 'silhouette']).T])
product_results

# Cluster analysis <a class="anchor" id="analysis"></a>

In [ ]:
def numerical_variables_analysis(df, labels_col, description):
  # calculate mean of each numerical variable per cluster
  numerical_means = df.groupby(labels_col, as_index=False).mean()

  plt.figure(figsize = (20, 7))

  pd.plotting.parallel_coordinates(numerical_means, labels_col, color = ['#25bbbf','#25bf5d', '#de3838', '#e5d627', '#2367c4', '#ad487f', '#de7e4e'])

  plt.suptitle(f'Cluster analysis for numerical variables following a {description} clustering perspective', fontsize = 20)
  plt.legend(title = 'Cluster')

  plt.show()

In [ ]:
def categorical_variables_analysis(df, labels_col, categorical_vars, description):
  fig, axes = plt.subplots(1, 3, figsize=(30, 7))

  for ax, var in zip(axes, categorical_vars):

    df_percent = df.groupby(labels_col)[var].value_counts(normalize = True).reset_index(name = 'percentage')
    sns.barplot(data = df_percent, x = labels_col, y = 'percentage', hue = var, ax = ax)

    ax.set_title(var + '% per cluster')
    ax.set_ylabel('Percentage')
    ax.set_xlabel('Cluster')


  plt.suptitle(f"Analysis of categorical variables for each cluster following a {description} prespective", fontsize = 20)

## Socio-Demographic <a class="anchor" id="sociodemoAnalysis"></a>

In [ ]:
df_clusters = df_scaled.copy()

In [ ]:
sociodemo_results

The best algorithm was K-Medoids

In [ ]:
distance_matrix = gower.gower_matrix(df_demo, cat_features = [False, True, True])
km = kmedoids.KMedoids(5, method='fasterpam', random_state = 0).fit(distance_matrix)
df_clusters['demo_labels'] = km.labels_

In [ ]:
df_clusters['demo_labels'].value_counts()

In [ ]:
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal', 'CustMonVal', 'ClaimsRate', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork', 'total_premiums', 'total_reversals']
categorical_features = ['GeoLivArea', 'EducDeg', 'Children']
numerical_variables_analysis(df_clusters[metric_features + ['demo_labels']], 'demo_labels', 'Socio-Demographic')

In [ ]:
categorical_variables_analysis(df_clusters[categorical_features + ['demo_labels']], 'demo_labels', categorical_features, 'demographic')

## Value <a class="anchor" id="valueAnalysis"></a>

In [ ]:
value_results

K-Means shows the best solution

In [ ]:
kmclust = KMeans(n_clusters = 4, init = 'k-means++', n_init = 15, random_state = 1)
labels = kmclust.fit_predict(df_value)
df_clusters['value_labels'] = labels

In [ ]:
df_clusters['value_labels'].value_counts()

In [ ]:
numerical_variables_analysis(df_clusters[metric_features + ['value_labels']], 'value_labels', 'value')

In [ ]:
categorical_variables_analysis(df_clusters[categorical_features + ['value_labels']], 'value_labels', categorical_features, 'value')

## Product <a class="anchor" id="productAnalysis"></a>

In [ ]:
product_results

The set of total premiums has better overall performance, thus we will discard the second set. The best algorithm is K-Means again

In [ ]:
kmclust = KMeans(n_clusters = 3, init = 'k-means++', n_init = 15, random_state = 1)
df_clusters['product_premiums_labels'] = kmclust.fit_predict(df_product_1)

In [ ]:
df_clusters['product_premiums_labels'].value_counts()

In [ ]:
numerical_variables_analysis(df_clusters[metric_features + ['product_premiums_labels']], 'product_premiums_labels', 'product (total premiums)')

In [ ]:
categorical_variables_analysis(df_clusters[categorical_features + ['product_premiums_labels']], 'product_premiums_labels', categorical_features, 'Product (Total Premiums)')

# Merging the perspectives <a class="anchor" id="merging"></a>

In [ ]:
# features used for clustering
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork', 'total_premiums']

feats = metric_features + categorical_features

# Centroids of the concatenated cluster labels
df_combinations = df_clusters .groupby(['demo_labels', 'value_labels', 'product_premiums_labels']).agg({c: 'mean' if c in metric_features else st.mode for c in feats})

df_combinations[categorical_features] = df_combinations[categorical_features].applymap(lambda x: x[0][0] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], list) and len(x[0]) > 0 else None)

df_combinations

In [ ]:
df_combinations.shape

We end up with 55 clusters. Let's merge them

In [ ]:
clusters_distance_matrix = gower.gower_matrix(df_combinations, cat_features = [False, False, False, False, False, False, False, False, False, True, True, True])

In [ ]:
inertia = []
silhouette = []
for n_clus in range(1, 11):
  km = kmedoids.KMedoids(n_clus, method='fasterpam', random_state = 8).fit(clusters_distance_matrix)
  inertia.append(km.inertia_)

  if n_clus == 1:
    continue
  labels = km.labels_
  silhouette.append(silhouette_score(clusters_distance_matrix, labels))

plot_scores(inertia, silhouette, 'final Clusters', (1, 11))

Best number of clusters if 4

In [ ]:
kmedoids_scores = {}

km = kmedoids.KMedoids(4, method='fasterpam', random_state = 8).fit(clusters_distance_matrix)
final_labels = km.labels_
df_combinations['kmedoids_labels'] = final_labels

print('Silhouette score', silhouette_score(clusters_distance_matrix, final_labels))

## Final clusters profiling <a class="anchor" id="profiling"></a>

In [ ]:
kmedoids_cluster_mapper = df_combinations['kmedoids_labels'].to_dict()

df_clusters['merged_labels'] = df_clusters.apply(lambda r: kmedoids_cluster_mapper[(r['demo_labels'], r['value_labels'], r['product_premiums_labels'])], axis = 1)

In [ ]:
df_clusters['merged_labels'].value_counts()

In [ ]:
distance_matrix = gower.gower_matrix(df_clusters[feats], cat_features = [False, False, False, False, False, False, False, False, False, True, True, True])

final_silhouette = silhouette_score(distance_matrix, df_clusters['merged_labels'])
print('Final solution silhouette score:', final_silhouette)

In [ ]:
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal', 'CustMonVal', 'ClaimsRate', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork', 'total_premiums', 'total_reversals']
numerical_variables_analysis(df_clusters[metric_features + ['merged_labels']], 'merged_labels', 'final')

In [ ]:
categorical_variables_analysis(df_clusters[categorical_features + ['merged_labels']], 'merged_labels', categorical_features, 'final')

In [ ]:
df_bubbles = df_clusters.groupby('merged_labels').agg({c : 'mean' if c in ['MonthSal', 'total_premiums', 'Children'] else 'size' for c in ['MonthSal', 'total_premiums', 'Children', 'BirthYear']})

In [ ]:
df_bubbles['Cluster'] = df_bubbles.index
df_bubbles.rename(columns = {'BirthYear' : 'Size'}, inplace = True)
df_bubbles['NoChildren'] = 1 - df_bubbles['Children']
df_bubbles

In [ ]:
def draw_pie(dist, xpos, ypos, size, ax, cluster):

    # for incremental pie slices
    cumsum = np.cumsum(dist)
    cumsum = cumsum/ cumsum[-1]
    pie = [0] + cumsum.tolist()

    colors = ['#25bbbf', '#de7e4e']

    for i, (r1, r2) in enumerate(zip(pie[:-1], pie[1:])):
        angles = np.linspace(2 * np.pi * r1, 2 * np.pi * r2)
        x = [0] + np.cos(angles).tolist()
        y = [0] + np.sin(angles).tolist()

        xy = np.column_stack([x, y])

        ax.scatter([xpos], [ypos], marker=xy, s=size * 2, color = colors[i], alpha = 0.7)
    
    ax.text(xpos - 0.015, ypos - 0.03, cluster, fontsize=15, fontname = 'Liberation Sans', color = '#444444')
    
    return ax

fig, ax = plt.subplots(figsize=(15,10))

cluster_0 = df_bubbles[df_bubbles['Cluster'] == 0]
cluster_1 = df_bubbles[df_bubbles['Cluster'] == 1]
cluster_2 = df_bubbles[df_bubbles['Cluster'] == 2]
cluster_3 = df_bubbles[df_bubbles['Cluster'] == 3]

i = 0
for cluster in [cluster_0, cluster_1, cluster_2, cluster_3]:
  draw_pie(cluster[['Children', 'NoChildren']].values.flatten().tolist(), cluster['MonthSal'], cluster['total_premiums'], cluster['Size'], ax=ax, cluster = i)
  i += 1


ax.set_facecolor("#e5ecf6")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

legend_elements = [plt.Line2D([0], [0], color = '#25bbbf', lw = 4, label='Yes'), plt.Line2D([0], [0], color = '#de7e4e', lw = 4, label='No')]
ax.legend(handles=legend_elements, loc='upper right', title = 'Children', title_fontsize = 15, fontsize = 12)

ax.set_axisbelow(True)
ax.yaxis.grid(color='white')
ax.xaxis.grid(color='white')

plt.title('Final clusters Bubble Chart', fontsize = 20, pad = 20)
plt.ylabel('total_premiums', fontsize = 15)
plt.xlabel('MonthSal', fontsize = 15)
plt.xlim(-1.5, 1)
plt.ylim(-1, 1.7)
plt.show()

## Visualizing the clusters with t-SNE <a class="anchor" id="tsne"></a>

In [ ]:
metric_features = ['FirstPolYear', 'BirthYear', 'MonthSal', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremLife', 'PremWork', 'total_premiums']

tsne = TSNE(n_components= 2, random_state=0)
projections = tsne.fit_transform(df_clusters[metric_features])

In [ ]:
plt.figure(figsize = (10, 7))
fig = px.scatter(
    projections, x=0, y=1,
    color=df_clusters['merged_labels'], labels={'color': 'Cluster'}, width=800, height=400,
    title = 'Clusters visualization using t-SNE'
)
fig.show()

## Silhouette of the final solution <a class="anchor" id="finalSilhouette"></a>

In [ ]:
plt.figure(figsize = (12, 8))

cluster_labels = df_clusters['merged_labels']
sample_silhouette_values = silhouette_samples(df_clusters[metric_features], cluster_labels)

y_lower = 10

for i in range(4):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sorted(sample_silhouette_values[cluster_labels == i])
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = len(ith_cluster_silhouette_values)
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / 4)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

# The vertical line for average silhouette score of all the values
plt.axvline(x=final_silhouette, color="red", linestyle="--")

# The silhouette coefficient can range from -1, 1
xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
plt.xlim([xmin, xmax])

plt.title('Silhouette plot for the final clusters')
plt.xlabel('Silhouette score values')
plt.yticks(color='w')

# Features importance <a class="anchor" id="featsImportance"></a>

In [ ]:
X = df_clusters[feats]
y = df_clusters['merged_labels']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state = 0)

clf = DecisionTreeClassifier(random_state = 0, max_depth = 5)

clf.fit(X_train, y_train)
print("Customers' cluster predicitions score:", np.round(clf.score(X_test, y_test)*100, 2))

In [ ]:
pd.Series(clf.feature_importances_, index=X_train.columns, name = 'Features Importance')

# Reclassifying outliers <a class="anchor" id="classifOutliers"></a>

In [ ]:
df_discarded.isna().sum()

The Decision Tree can not deal with missing values, so we first need to fill them. There is just one entry that has missing values, so let's fill them using measures of central tendency

In [ ]:
df_discarded['FirstPolYear'].fillna(df_discarded['FirstPolYear'].median(), inplace = True)
df_discarded['GeoLivArea'] = df_discarded['GeoLivArea'].fillna(df_discarded['GeoLivArea'].mode().values[0])
df_discarded['Children'].fillna(df_discarded['Children'].mode().values[0], inplace = True)

df_discarded.isna().sum()

Finally, we get the clusters to which each discarded observation (incoherences or outliers) belongs to.

In [ ]:
df_discarded['merged_labels'] = clf.predict(df_discarded[feats])
df_discarded.head()

Now we have all customers belonging to a cluster. Therefore, we can use targeted marketing strategies on them.